In [1]:
import or_gym
from or_gym.benchmarks.newsvendor.nv_benchmark import *
from or_gym.benchmarks.newsvendor.onv_benchmark import *
import matplotlib.pyplot as plt
import pyomo.environ as pe

In [2]:
#create environments for 5 cases
env1_dfo=or_gym.make("NewsVendor-v1")#,env_config={'periods':20})
env1_mip=or_gym.make("NewsVendor-v1")#,env_config={'periods':20})
env1_mip_pi=or_gym.make("NewsVendor-v1")#,env_config={'periods':20})
env1_dfo_online=or_gym.make("NewsVendor-v1")#,env_config={'periods':20})
env1_mip_online=or_gym.make("NewsVendor-v1")#,env_config={'periods':20})

C:\ProgramData\Miniconda3\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [3]:
#run optimizations for backlog scenario
env1_dfo_results = optimize_nv_dfo(env1_dfo)

   direc: array([[   1.        ,    0.        ,    0.        ],
       [   0.        ,    1.        ,    0.        ],
       [-242.43186687,  110.48948914,  164.23304955]])
    fopt: 0.5353396383764859
     fun: -0.535444877561173
 message: 'Optimization terminated successfully.'
    nfev: 515
     nit: 8
  status: 0
 success: True
       x: array([ 81.99986982, 115.00000333, 216.14822517])
    xopt: array([ 82., 115., 217.])
    zopt: array([ 82., 197., 414.])


In [ ]:
env1_mip_model, env1_mip_results = optimize_nv_mip(env1_mip,warmstart=True,
                                                   warmstart_kwargs={'mapping_env': env1_dfo,
                                                                     'mapping_z':env1_dfo_results.zopt})

Academic license - for non-commercial use only
Read LP format model from file C:\Users\HD\AppData\Local\Temp\tmpax2u_k0e.pyomo.lp
Reading time = 0.03 seconds
x3343: 4147 rows, 2980 columns, 15865 nonzeros
Read MIP start from file C:\Users\HD\AppData\Local\Temp\tmpj5a42wxc.gurobi.mst
Optimize a model with 4147 rows, 2980 columns and 15865 nonzeros
Variable types: 1897 continuous, 1083 integer (1080 binary)
Coefficient statistics:
  Matrix range     [2e-03, 1e+04]
  Objective range  [4e-05, 1e-03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+04]

Loaded MIP start with objective 0.53534

Presolve removed 1235 rows and 765 columns
Presolve time: 0.03s
Presolved: 2912 rows, 2215 columns, 10827 nonzeros
Variable types: 1233 continuous, 982 integer (977 binary)

Root relaxation: objective 8.155037e-01, 2277 iterations, 0.04 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Nod

In [ ]:
env1_mip_pi_model, env1_mip_pi_results = optimize_nv_pi_mip(env1_mip_pi,warmstart=True,
                                                   warmstart_kwargs={'mapping_env': env1_dfo,
                                                                     'mapping_z':env1_dfo_results.zopt,
                                                                     'perfect_information':True})

In [ ]:
_,_,env1_dfo_online_basestock = online_optimize_nv_dfo(env1_dfo_online)

In [ ]:
_,_,env1_mip_online_basestock = online_optimize_nv_mip(env1_mip_online,warmstart=True)

In [ ]:
#plot results for backlog scenario

#profit
fig = plt.figure(figsize=[7,5])
plt.plot(np.cumsum(env1_dfo.P),label='DFO')
plt.plot(np.cumsum(env1_mip.P),label='MIP')
plt.plot(np.cumsum(env1_dfo_online.P),label='oDFO')
plt.plot(np.cumsum(env1_mip_online.P),label='oMIP')
plt.plot(np.cumsum(env1_mip_pi.P),'k',label='Oracle')
plt.xlabel('Day')
plt.ylabel('Cumulative Profit, $')
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
plt.tight_layout()
plt.savefig('NewsVendor_Backlog.png')
plt.show()

#base stock levels
N = env1_dfo.num_periods
M = env1_dfo.num_stages
env1_dfo_online_basestock = np.array(env1_dfo_online_basestock)
env1_mip_online_basestock = np.array(env1_mip_online_basestock)

def inventory_position(I,T,B,stage):
    return np.sum(I[:-1,:stage+1] + T[:-1,:stage+1] - np.row_stack((np.zeros(stage+1),B[:-1,:i+1])),axis=1)

fig, axs = plt.subplots(3,1,figsize=[7,13])
axs = axs.ravel()
for i in range(M-1):
    axs[i].set_prop_cycle(None)
    axs[i].set_title('Echelon %i' %i)
    axs[i].set_xlabel('Day')

    #plot base stock levels
#     axs[i].plot(env1_dfo_results.zopt[i]*np.ones(N),label='DFO BSL')
#     axs[i].plot(pe.value(env1_mip_model.z[i])*np.ones(N),label='MIP BSL')
#     axs[i].plot(np.append(env1_dfo_online.init_inv[i],env1_dfo_online_basestock[:,i]),label='oDFO BSL')
#     axs[i].plot(np.append(env1_mip_online.init_inv[i],env1_mip_online_basestock[:,i]),label='oMIP BSL')
    
    axs[i].set_prop_cycle(None)
    #plot actual inventory positions
    axs[i].plot(inventory_position(I = env1_dfo.I, T = env1_dfo.T, B = env1_dfo.B, stage = i),'--',label='DFO')
    axs[i].plot(inventory_position(I = env1_mip.I, T = env1_mip.T, B = env1_mip.B, stage = i),':',label='MIP')
    axs[i].plot(inventory_position(I = env1_dfo_online.I, T = env1_dfo_online.T, B = env1_dfo_online.B, stage = i),'-.',label='oDFO')
    axs[i].plot(inventory_position(I = env1_mip_online.I, T = env1_mip_online.T, B = env1_mip_online.B, stage = i),'-.',label='oMIP')
    axs[i].plot(inventory_position(I = env1_mip_pi.I, T = env1_mip_pi.T, B = env1_mip_pi.B, stage = i),'k',label='Oracle')
    axs[i].legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
plt.tight_layout()
plt.savefig('NewsVendor_Backlog_inventory.png')
plt.show()

fig, axs = plt.subplots(3,1,figsize=[7,13])
axs = axs.ravel()
for i in range(M-1):
    axs[i].set_prop_cycle(None)
    axs[i].set_title('Echelon %i' %i)
    axs[i].set_xlabel('Day')

    #plot base stock levels
    axs[i].plot(env1_dfo_results.zopt[i]*np.ones(N),label='DFO')
    axs[i].plot(pe.value(env1_mip_model.z[i])*np.ones(N),label='MIP')
    axs[i].plot(np.append(env1_dfo_online.init_inv[i],env1_dfo_online_basestock[:,i]),label='oDFO')
    axs[i].plot(np.append(env1_mip_online.init_inv[i],env1_mip_online_basestock[:,i]),label='oMIP')
    
#     axs[i].set_prop_cycle(None)
#     #plot actual inventory positions
#     axs[i].plot(inventory_position(I = env1_dfo.I, T = env1_dfo.T, B = env1_dfo.B, stage = i),'--',label='DFO TIL')
#     axs[i].plot(inventory_position(I = env1_mip.I, T = env1_mip.T, B = env1_mip.B, stage = i),':',label='MIP TIL')
#     axs[i].plot(inventory_position(I = env1_dfo_online.I, T = env1_dfo_online.T, B = env1_dfo_online.B, stage = i),'-.',label='oDFO TIL')
#     axs[i].plot(inventory_position(I = env1_mip_online.I, T = env1_mip_online.T, B = env1_mip_online.B, stage = i),'-.',label='oMIP TIL')
#     axs[i].plot(inventory_position(I = env1_mip_pi.I, T = env1_mip_pi.T, B = env1_mip_pi.B, stage = i),'k',label='Oracle TIL')
    axs[i].legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
plt.tight_layout()
plt.savefig('NewsVendor_Backlog_zopt.png')
plt.show()
                
# #on hand inventory levels
# env1_dfo_online_basestock = np.array(env1_dfo_online_basestock)
# env1_mip_online_basestock = np.array(env1_mip_online_basestock)
# fig, axs = plt.subplots(3,1,figsize=[5,10])
# axs = axs.ravel()
# for i in range(M-1):
#     axs[i].set_title('Stage %i' %i)
#     axs[i].set_xlabel('Day')
#     axs[i].plot(env1_dfo.I[:,i],label='DFO')
#     axs[i].plot(np.reshape(list(env1_mip_model.I.get_values().values()),(N+1,env1_dfo.num_stages-1))[:,i],label='MIP')
#     axs[i].plot(env1_dfo_online.I[:,i],label='oDFO')
#     axs[i].plot(env1_mip_online.I[:,i],label='oMIP')
#     axs[i].legend()
# plt.tight_layout()
# plt.show()